In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)

%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from numpy.random import randn
from numpy import diag, ndarray



In [3]:
import sys
sys.path.append('..')

In [5]:
from utils.kronecker import KroneckerOperator, KroneckerProduct, KroneckerSum, KroneckerDiag, _ProductChain, _SumChain
from utils.linalg import kronecker_product_literal, kronecker_sum_literal, vec, ten


In [6]:
N1 = 6
N2 = 5
N3 = 4
N4 = 3

A1 = randn(N1, N1)
A2 = randn(N2, N2)
A3 = randn(N3, N3)
A4 = randn(N4, N4)

X = randn(N4, N3, N2, N1)
D = randn(N4, N3, N2, N1)

In [9]:
vec(X) @ kronecker_product_literal(A1, A2, A3, A4)

array([-10.592,  15.608,  -3.793,   0.324,  -1.658,   7.715, -16.8  ,  26.086,  -5.061, -10.498,  14.52 ,  -4.342, -10.958,  17.332,  -0.288,   2.437,  -4.392,   0.924,  -9.579,  13.645,   0.567,  -9.483,  13.674,   0.145, -10.158,  14.96 ,  -7.077,  -0.008,  -0.707,   0.026,  -9.035,  11.659,  -4.755, -10.197,  14.898,  -5.806, -25.729,  35.121,  -1.336, -12.182,  19.375,  -0.003, -19.665,  23.341,   2.457, -13.38 ,  14.805,   1.956,  -7.186,  11.913,  -0.312,   3.222,  -6.355,   5.674,
       -11.189,  18.225,  -2.07 ,  -7.94 ,  11.821,  -1.662,  -0.469,   3.272,  -3.157,   1.509,  -5.513,   5.966,  -6.418,  13.661,  -1.1  ,  -0.546,   1.922,  -1.511, -13.579,  23.846, -11.956,   6.724,  -9.556, -11.503, -11.428,  19.965,  -9.78 , -14.801,  24.264,  -5.006, -13.499,  15.705,  -3.698,   0.598,  -2.609, -10.691, -11.7  ,  14.296,  -0.012, -12.783,  17.648,   0.393, -12.916,  14.006,  -8.709,   7.223,  -8.389,   2.813, -13.33 ,  15.192,   2.601, -20.792,  23.725,   0.247,
        -6.771

In [32]:
class Dummy(object):
    
#     this_does_not_exist = 5
    
#     def __getattr__(self, attr):
#         print('GETTTING')
#         return attr.upper() 
    
    def __getattribute__(self, name):
        self.hi = 'hi'
        print(name)
        return object.__getattribute__(self, name)
    
d = Dummy()
d.this_does_not_exist


this_does_not_exist


AttributeError: 'Dummy' object has no attribute 'this_does_not_exist'

In [58]:
class Yeah(object):
    def __init__(self, name):
        
        self.name = name    
        
    def __getattribute__(self, item):
        
        print(' getting property {}'.format(item))
        
        return object.__getattribute__(self, item)
 
    


In [60]:
Y = Yeah('Ed')

__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __

__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __

RecursionError: maximum recursion depth exceeded

In [50]:
Y.E

__getattribute__  E
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__
__getattribute__  __class__


RecursionError: maximum recursion depth exceeded